<a href="https://colab.research.google.com/github/KOMBOU12/Marius/blob/main/NLP_R%C3%A9sum%C3%A9_d'article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voici l'implémentation que j'ai crée qui divise les articles avant de les résumer.

Mais ça prend trop de temps pour être exécuter sur les 400 articles du train set OBS.

Peut-être l'essayer sur 10 articles ne serait pas une mauvaise idée.

Ici le modèle est déjà préentrainé.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
import pandas as pd

# Charger le modèle LED pour résumer des textes longs
model_name = "allenai/led-large-16384"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Dossier contenant les fichiers d'articles
dossier_articles = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/OBS/articles_OBS/"

# Fonction pour lire un fichier
def lire_article(chemin_dossier, nom_fichier):
    chemin_complet = os.path.join(chemin_dossier, nom_fichier)
    with open(chemin_complet, 'r', encoding='utf-8') as fichier:
        return fichier.read()

# Fonction pour résumer un texte long
def resumer_texte_long(texte, max_length=512, chunk_size=4096):
    # Diviser le texte en morceaux
    morceaux = [texte[i:i + chunk_size] for i in range(0, len(texte), chunk_size)]
    resumes = []
    for morceau in morceaux:
        # Générer un résumé pour chaque morceau
        inputs = tokenizer(morceau, return_tensors="pt", truncation=True, max_length=chunk_size)
        outputs = model.generate(inputs["input_ids"], max_length=max_length, min_length=50, length_penalty=2.0)
        resumes.append(tokenizer.decode(outputs[0], skip_special_tokens=True))

    # Combiner les résumés partiels en un seul texte
    texte_final = " ".join(resumes)
    return texte_final

# Lister tous les fichiers d'articles
fichiers_articles = [f for f in os.listdir(dossier_articles) if f.startswith("article-")]

# Liste pour stocker les résumés générés
resumes_generes = []

# Traiter chaque fichier d'article
for article_fichier in fichiers_articles:
    try:
        # Lire le contenu de l'article
        contenu_article = lire_article(dossier_articles, article_fichier)

        # Résumer l'article
        resume = resumer_texte_long(contenu_article, max_length=512, chunk_size=4096)

        # Ajouter le résumé et l'identifiant à la liste
        resumes_generes.append({
            "fichier": article_fichier,
            "resume": resume
        })

        # Afficher un message de progression
        print(f"Résumé généré pour : {article_fichier}")

    except Exception as e:
        print(f"Erreur pour l'article {article_fichier}: {e}")

# Sauvegarder les résumés dans un fichier CSV
df_resumes = pd.DataFrame(resumes_generes)
df_resumes.to_csv("resumes_articles.csv", index=False, encoding="utf-8")
print("Tous les résumés ont été sauvegardés dans 'resumes_articles.csv'")


**Importationn du modèle préentrainé**

In [ ]:
import os
import random
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import evaluate
import gc

hf_token = "" #Mettre son token dans Kaggle

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
    token=hf_token
)

**Partie du code pour résumer 10 articles du dossier test et pour obtenir les score**

In [ ]:
import os
import random
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import evaluate
import gc



articles_dir = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/test/OBS_test/articles_OBS_test/"
article_files = [f for f in os.listdir(articles_dir) if f.endswith('.txt')]
selected_articles = random.sample(article_files, 10)

summaries = []
rouge = evaluate.load("rouge")

for article_file in selected_articles:
    article_path = os.path.join(articles_dir, article_file)
    with open(article_path, "r", encoding="utf-8") as file:
        article_text = file.read()


    prompt =[
    {"role":"system", "content" : "You are an expert assistant specialized in summarizing scientific articles. Provide a structured summary"},
     {"role":"user", "content" : article_text}]
    chat = tokenizer.apply_chat_template(prompt, tokenize=False)

    input_ids = tokenizer(chat, add_special_tokens=True, return_tensors="pt").to("cuda")

    try:
        output_ids = model.generate(
            **input_ids, #input_ids["input_ids"],
            #attention_mask=input_ids["attention_mask"],
            do_sample=True,
            top_p=50,  # Réduction pour privilégier les options les plus probables
            temperature=1,  # Réduction pour limiter la stochasticité
            max_new_tokens=150  # Plus de tokens pour des résumés plus longs
        )
        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    except torch.cuda.OutOfMemoryError:
        print(f"Erreur de mémoire pour {article_file}. Réduction des tokens...")
        torch.cuda.empty_cache()
        gc.collect()
        continue

    summaries.append({"Article_ID": article_file, "Summary": summary})

    # Remplacez par un vrai résumé de référence
    reference_summary = "Résumé réel de l'article ici."
    results = rouge.compute(predictions=[summary], references=[reference_summary])

    print(f"Scores ROUGE pour {article_file}:")
    for key, value in results.items():
        print(f"  {key}: {value}")

    torch.cuda.empty_cache()
    gc.collect()


submission_path = "/kaggle/working/submission2.csv"
submission2 = pd.DataFrame(summaries)
submission2.to_csv(submission_path, index=False)

Vous pouvez essayer le modèle :  **meta-llama/Llama-3.1-8B-Instruct** . Je n'ai plus d'heures de GPU pour continuer.

Ce modèle a 8 milliards de paramètres contre 3 milliards pour le modèle précédent.

In [ ]:
hf_token = " "

# Charger le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
    token=hf_token
)

# Tester un résumé avec le nouveau modèle
for article_file in selected_articles:
    article_path = os.path.join(articles_dir, article_file)
    with open(article_path, "r", encoding="utf-8") as file:
        article_text = file.read()

    prompt = [
        {"role": "system", "content": "You are an expert assistant specialized in summarizing scientific articles. Provide a structured summary."},
        {"role": "user", "content": article_text}
    ]
    chat = tokenizer.apply_chat_template(prompt, tokenize=False)

    input_ids = tokenizer(chat, add_special_tokens=True, return_tensors="pt").to("cuda")

    try:
        output_ids = model.generate(
            **input_ids,
            do_sample=True,
            top_p=0.8,
            temperature=0.5,
            max_new_tokens=200  # Alloué plus de tokens pour des résumés plus longs
        )
        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    except torch.cuda.OutOfMemoryError:
        print(f"Erreur de mémoire pour {article_file}. Réduction des tokens...")
        torch.cuda.empty_cache()
        gc.collect()
        continue

    summaries.append({"Article_ID": article_file, "Summary": summary})

    # ROUGE : à évaluer avec un vrai résumé de référence
    reference_summary = "Résumé réel de l'article ici."
    results = rouge.compute(predictions=[summary], references=[reference_summary])

    print(f"Scores ROUGE pour {article_file}:")
    for key, value in results.items():
        print(f"  {key}: {value}")

    torch.cuda.empty_cache()
    gc.collect()

# Sauvegarder les résultats
submission_path = "/kaggle/working/submission8B.csv"
submission = pd.DataFrame(summaries)
submission.to_csv(submission_path, index=False)


# **Le fameux BART !!!!**

Cette partie initialise les composants nécessaires pour générer des résumés avec BART.

In [ ]:
import random
from datasets import Dataset
from transformers import BartForConditionalGeneration, BartTokenizer
from rouge_score import rouge_scorer
import torch

# Charge le modèle BART et le tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

Ici on utilise le modèle préentraîné BART pour générer des résumés à partir d'articles scientifiques, on les compare avec des résumés de référence, et on calcule le score ROUGE-2 pour évaluer la qualité des résumés générés.

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
from rouge_score import rouge_scorer
import os
import random

# Charger le modèle BART et le tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Chemins vers les dossiers des articles et des résumés de référence
dossier_articles = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/OBS/articles_OBS/"
dossier_resumes = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/OBS/abstracts_OBS/"

# Charger les articles et leurs résumés de référence
fichiers_articles = [f for f in os.listdir(dossier_articles) if f.startswith("article-")]
articles = {}
resumes_reference = {}

def lire_fichier(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read()
    except UnicodeDecodeError:
        with open(filepath, 'r', encoding='latin-1') as f:
            return f.read()

for fichier in fichiers_articles:
    identifiant = fichier.split("-")[1].split(".")[0]
    articles[identifiant] = lire_fichier(os.path.join(dossier_articles, fichier))
    resumes_reference[identifiant] = lire_fichier(os.path.join(dossier_resumes, f"abstract-{identifiant}.txt"))

# Sélectionner 5 articles aléatoires
random.seed(42)  # Pour garantir la reproductibilité
selected_ids = random.sample(list(articles.keys()), 5)

# Stocker les articles, résumés générés et scores
resumes_generes = []
rouge_scores = []

# Fonction pour résumer un texte avec BART
def resumer_texte(texte, max_length=130, min_length=30, length_penalty=2.0, num_beams=4):
    inputs = tokenizer.encode("summarize: " + texte, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs,
        max_length=max_length,
        min_length=min_length,
        length_penalty=length_penalty,
        num_beams=num_beams,
        early_stopping=True,
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Calcul des résumés et des scores ROUGE-2
scorer = rouge_scorer.RougeScorer(["rouge2"], use_stemmer=True)

for identifiant in selected_ids:
    article = articles[identifiant]
    resume_reference = resumes_reference[identifiant]

    # Générer le résumé
    resume_genere = resumer_texte(article)
    resumes_generes.append((identifiant, resume_genere))

    # Calculer le score ROUGE-2
    scores = scorer.score(resume_reference, resume_genere)
    rouge_scores.append(scores["rouge2"].fmeasure)

# Afficher les résultats
for i, (identifiant, resume_genere) in enumerate(resumes_generes):
    print(f"Article ID: {identifiant}")
    print(f"Résumé généré: {resume_genere}")
    print(f"ROUGE-2 Score: {rouge_scores[i]:.4f}")
    print("-" * 50)

# Afficher le score ROUGE-2 moyen
avg_rouge2 = sum(rouge_scores) / len(rouge_scores)
print("Score ROUGE-2 moyen:")
print(f"ROUGE-2: {avg_rouge2:.4f}")


On a une moyenne de score de 0,04. **BART** a une limite de 1024 tokens. Les articles plus longs sont tronqués, ce qui peut entraîner une perte d'informations essentielles. Les résumés générés pourraient être incomplets ou manquer de cohérence.


# **Modèle utilisant un Prompt, score moyen de 0,12 sur 10 articles**

Ce modèle utilise un prompt, prendes articles les resummes , ensuite calcul le Score Rouge 2 pour chaque article, somme les scores et donne la moyenne.

In [ ]:
import os
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import evaluate
import gc

# Charger le tokenizer et le modèle
hf_token = "" #Mettre ton Token à toi
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
    token=hf_token
)

# Chemins des articles et résumés de référence
dossier_articles = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/OBS/articles_OBS/"
dossier_resumes = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/OBS/abstracts_OBS/"

# Charger les fichiers
article_files = [f for f in os.listdir(dossier_articles) if f.endswith('.txt')]
selected_articles = random.sample(article_files, 10)

# Charger la métrique ROUGE
rouge = evaluate.load("rouge")

# Calculer les scores ROUGE-2
rouge_scores = []

for article_file in selected_articles:
    article_path = os.path.join(dossier_articles, article_file)
    identifiant = article_file.split("-")[1].split(".")[0]
    resume_path = os.path.join(dossier_resumes, f"abstract-{identifiant}.txt")

    try:
        # Lire l'article et le résumé de référence
        with open(article_path, "r", encoding="utf-8") as file:
            article_text = file.read()
        with open(resume_path, "r", encoding="utf-8") as file:
            reference_summary = file.read()

        # Préparer l'invite pour le modèle
        prompt = (
            f"System: You are an expert assistant. Provide a clear, concise summary for the following scientific article.\n"
            f"Article:\n{article_text[:3000]}\nSummary:"
        )

        # Tokenisation de l'invite
        input_ids = tokenizer(prompt, add_special_tokens=True, return_tensors="pt", truncation=True, max_length=1024).to("cuda")

        # Génération du résumé
        output_ids = model.generate(
            input_ids=input_ids["input_ids"],
            attention_mask=input_ids["attention_mask"],
            do_sample=True,
            top_p=0.85,
            temperature=0.5,
            max_new_tokens=300
        )
        generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Calcul du score ROUGE-2
        results = rouge.compute(predictions=[generated_summary], references=[reference_summary])
        rouge_scores.append(results["rouge2"])

    except Exception as e:
        print(f"Erreur pour l'article {article_file}: {e}")

    # Libérer la mémoire GPU
    torch.cuda.empty_cache()
    gc.collect()

# Afficher les scores ROUGE-2
print("Scores ROUGE-2 pour les articles sélectionnés :")
for score in rouge_scores:
    print(f"ROUGE-2: {score:.4f}")

# Calculer et afficher la moyenne
avg_rouge2 = sum(rouge_scores) / len(rouge_scores) if rouge_scores else 0
print(f"Score ROUGE-2 moyen : {avg_rouge2:.4f}")


# **MISE A JOUR DU MODELE BART**

Dans le moèle précedent les appariements n'ont pas été pris en compte.

In [ ]:
import random
from datasets import Dataset
from transformers import BartForConditionalGeneration, BartTokenizer
from rouge_score import rouge_scorer
import torch

# Charge le modèle BART et le tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

In [ ]:
import os
from transformers import BartForConditionalGeneration, BartTokenizer
from rouge_score import rouge_scorer

# Initialisation du modèle et du tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Chemins vers les dossiers des articles et résumés
dossier_abstracts = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/OBS/abstracts_OBS/"
dossier_articles = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/OBS/articles_OBS/"

# Fonction pour lire un fichier avec gestion des erreurs d'encodage
def lire_fichier(chemin_fichier):
    try:
        with open(chemin_fichier, "r", encoding="utf-8") as f:
            return f.read()
    except UnicodeDecodeError:
        with open(chemin_fichier, "r", encoding="latin-1") as f:
            return f.read()

# Fonction pour générer un résumé avec BART
def resumer_texte(texte, max_length=130, min_length=30, length_penalty=2.0, num_beams=4):
    inputs = tokenizer.encode("summarize: " + texte, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs,
        max_length=max_length,
        min_length=min_length,
        length_penalty=length_penalty,
        num_beams=num_beams,
        early_stopping=True,
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Initialisation du scoreur ROUGE
scorer = rouge_scorer.RougeScorer(["rouge2"], use_stemmer=True)

# Application du modèle et calcul des scores ROUGE-2
rouge_scores = []
for i in range(5):  # Lire les 5 premiers articles
    identifiant = dataset[i]["identifiant"]
    fichier_article = dataset[i]["article"]
    fichier_abstract = dataset[i]["abstract"]

    # Chemins complets des fichiers
    chemin_fichier_article = os.path.join(dossier_articles, fichier_article)
    chemin_fichier_abstract = os.path.join(dossier_abstracts, fichier_abstract)

    # Lire le contenu des fichiers
    contenu_article = lire_fichier(chemin_fichier_article)
    contenu_abstract = lire_fichier(chemin_fichier_abstract)

    # Générer le résumé avec le modèle
    resume_genere = resumer_texte(contenu_article)

    # Calculer le score ROUGE-2
    rouge_score = scorer.score(contenu_abstract, resume_genere)["rouge2"].fmeasure
    rouge_scores.append(rouge_score)

    # Afficher les résultats pour cet article
    print(f"Article ID: {identifiant}")
    print(f"Résumé généré:\n{resume_genere}")
    print(f"Résumé réel:\n{contenu_abstract}")
    print(f"Score ROUGE-2: {rouge_score:.4f}")
    print("-" * 80)

# Calculer et afficher le score ROUGE-2 moyen
avg_rouge2 = sum(rouge_scores) / len(rouge_scores)
print(f"Score ROUGE-2 moyen sur 5 articles : {avg_rouge2:.4f}")


Le score est toujours nul mais légèrement en augmentation, comme quoi les appraiements ont un impacte.